# Basics of mobile robotics project - The Thytanic Navigation (2024)

## 1. Introduction & Context

| **Contributors**         | **Sciper**  | **Role**             | **Studies**                                                                   |
|---------------------------|-------------|-----------------------|---------------------------------------------------------------------------  |
| **Alessio Desogus**       | 301705      | `Local Navigation`      | Bachelor and Master in Mechanical Engineering                              |
|   **Antoine Bachmann**    |  336641     | `Global navigation`      | Bachelor and Master in Computer Science                                   |
|**Ramon Heeb**             |   396213    | `Vision`                | Bachelor in Computer Science, Master in Robotics                           |
| **Adélaïde Pinel**        |330753       |  `Kalman Filtering`                     |  Bachelor and Master in Mechanical Engineering             |

Date of deliverable: 05.12.2024  

Our main sources were: 
- .
- .

### Project Description

This project involves guiding the **Thytanic** from its starting position to its destination in an efficient manner using `Global Navigation`. Along the way, the Thytanic must avoid appearing obstacles by employing `Local Navigation`. Additionally, the position of the Thytanic is estimated using a `Kalman Filter`. Finally, a stationary camera is positioned above the map to provide `Vision` information for the system’s navigation modules.

Imagine the following scenario: the Thytanic is navigating in the vast ocean (**blue background**) and is seeking to reach land (**green area**) safely while avoiding dangerous icebergs (**white obstacles**). At the same time, it must avoid colliding with **black zones** representing dangerous land masses. Its goal is to navigate the shortest, most energy-efficient route to safety.


![Local Navigation Algorithm](img/thytanic.png)

- Image reference background: https://www.freepik.com/premium-ai-image/iceberg-water-with-icebergs-background_55113664.htm
- Image reference thymio: https://edu.ge.ch/site/desrobotsenclasse/wp-content/uploads/sites/269/2015/02/front.jpg


## 2. Initialization

We start by importing the necessary modules, functions and basic libraries:  
- `local_navigation_and_control_all` for the local navigation and overall control of the Thytanic  
- `vision` for computer vision functions  
- `global_navigation` for global path planning functions  
- `kalman_filtering_all` for Kalman filter functions

In [13]:
from local_navigation_and_control import ThytanicController, ThytanicState
from vision import get_current_state, init_cam
from global_navigation import downsamplingprep, downsampling, pathfinder
from kalman_filtering import kalman_filter
import numpy as np
import matplotlib.pyplot as plt
import time

Next, we initialize the `ThytanicController`: 

In [14]:
thytanic = ThytanicController()

## 3. Vision

In [15]:
#turn the image into a 4-channel map
cam = init_cam()
obstacle_range=[np.array([0, 130, 0]), np.array([160, 240, 140])]
target_range=[np.array([0, 30, 235]), np.array([255, 130, 256])]
th_front_range=[np.array([0, 160, 253]), np.array([15, 250, 256])]
th_back_range=[np.array([16, 130, 141]), np.array([40, 250, 252])]
img, _, _ = get_current_state(
    cam=cam,
    obstacle_range=obstacle_range,
    target_range=target_range,
    th_front_range=th_front_range,
    th_back_range=th_back_range,
)

## 4. Global Navigation


For global navigation, we have two distinct phases: image postprocessing and pathfinding.

The first turns the already cleaned and processed image into an obstacle position binary 2d array, a thymio position, and an objective position.

This is done using significant downsampling, as the thymio has limited precision in its movements and as such very small pixels would not bring significant utility while making all processing operations much more expensive.

- The 2d array is binary and only features obstacles and free spaces, this is for ease of morphological operations, as the other two data points can be summed up to a single point which is easier to keep out of array for programming purposes, especially testing.

- The thymio position on the image, and thus its equivalent in the output array uses the thimyo's pixels average position, this is due to ease of implementation and practical functionality, more complex solutions were not needed, so we went no further.
  
- The goal position is as the thymio position.


The 2d array is then dilated morphologically and its pixels fused to reach the desired size, the threshold parameter determines what minimum proportion of a superpixel should contain obstacles for the superpixel to have a value of 1.

Pathfinding is conceptually very simple, thanks to the limited size of our 2darray, dijkstra becomes computationally viable. As our system has no cul-de-sacs or significant detours, euclidean distance from the objective is an excellent heuristic which allows us to implement an [A*](https://dl.acm.org/doi/10.1145/3828.3830) algorithm that provides a very significant improvement in computation time.

Here we used the square of the euclidean distance once again for computational reasons, this gave extreme weight to the heuristic compared to the path length but as long as the obstacles remain cul-de-sac free this is a massive computational gain over djikstra.



In [16]:
endsize = [120, 160]

#this line generates the original enposition, thymio position, and the obstacle channel
thymiopos, endpos, obstacleimage = downsamplingprep(img, endsize, 1, 1)

#this line turns obstacleimage into its downsampled version with enlarged obstacles
preppedimage = downsampling(obstacleimage, endsize, 25, 0.2)


plt.imshow(preppedimage)
plt.plot()

<IPython.core.display.Javascript object>

[]

The three addition lines are here for some data visualization, the colours let us see what nodes were taken as path points, explored, and unexplored but considered by the algorithm.

In [17]:
path, exploredpoints, unexploredpoints = pathfinder(thymiopos, endpos, preppedimage)

testimage = preppedimage.copy()

for k in path:
    testimage[k[0], k[1]] += 2
    
for k in exploredpoints:
    testimage[k[0], k[1]] += 3

for k in unexploredpoints:
    testimage[k[0], k[1]] += 4


plt.imshow(testimage)
plt.plot()

<IPython.core.display.Javascript object>

[]

## 5. Kalman Filter

## 6. Local Navigation

For the local navigation of the Thytanic, we implemented a state machine with 3 distinct states:  
- `GLOBAL_MOVEMENT`  
- `AVOIDING_OBSTACLE`  
- `STOP`  

The navigation begins in the `GLOBAL_MOVEMENT` state, where the Thytanic follows the optimal path provided. When a proximity sensor (*prox.horizontal*) detects an obstacle based on a pre-defined threshold, the Thytanic transitions to the `AVOIDING_OBSTACLE` state. In this state, it initiates avoidance maneuvers by turning to navigate around the obstacle. 

Once the sensors no longer detect any obstacles, the Thytanic moves straight for 3 seconds to create distance from the obstacle. It then updates the next point on the optimal path by skipping ahead two steps to ensure efficiency and prevent revisiting the same obstacle. After completing these maneuvers, the Thytanic transitions back to the `GLOBAL_MOVEMENT` state, following its journey along the optimal path. The `STOP` state is triggered when the Thytanic reaches its goal, at which point it halts completely, stopping all motors.  

![Local Navigation Algorithm](img/local_navigation.png)

Reguarding the **kidnapping scenario**, which simulates the thymio being lifted and placed elsewhere, the event is detected using the accelerometer by identifying an unexpected force in the z-axis. Upon detection, the Thytanic immediately stops its movement, sets its wheel speeds to zero, and waits for 10 seconds to stabilize before recalibrating. During this recalibration, the robot re-evaluates its position using the camera which captures the surrounding environment. Based on this new information, a new optimal path is calculated from the updated position to the target. Once the new path is successfully computed, the Thytanic's state is switched back to `GLOBAL_MOVEMENT`. 

![Kidnapping Scenario](img/kidnapping_scenario.png)

- The mermaid charts were done using the following website: https://www.mermaidchart.com

## 7. Main Loop

In [18]:
#https://stackoverflow.com/questions/14261903/how-can-i-open-the-interactive-matplotlib-window-in-ipython-notebook
#https://stackoverflow.com/questions/35532498/animation-in-ipython-notebook
%matplotlib tk
# Main script to establish connection and start movement
try:
    fig = 0
    print("Connecting to Thytanic...")
    thytanic.establish_connection()
    print("Connection established!")
    _, thymiopos, initital_angle = get_current_state(    cam=cam,
    obstacle_range=[np.array([0, 150, 25]), np.array([30, 220, 100])],
    target_range=[np.array([0, 30, 235]), np.array([255, 130, 256])],
    th_front_range=[np.array([0, 160, 250]), np.array([15, 250, 256])],
    th_back_range=[np.array([16, 130, 150]), np.array([35, 250, 249])],)
    # Set the robot to GLOBAL_MOVEMENT state
    print("initital_angle:", initital_angle)
    thytanic.x_est = np.array([thymiopos[1], 0, thymiopos[0], 0,initital_angle,0]) #position, velocity (x,y,angle)
    thytanic.P_est = np.eye(6) # TO TUNE
    thytanic.goal_idx = 0
    thytanic.robot_state = ThytanicState.GLOBAL_MOVEMENT
    #thytanic.set_wheel_speed(100, 100)
    print("Thytanic state set to GLOBAL_MOVEMENT.")
    x_estimated = []
    ########################
    
    path = path[::5]
    path = path[2:]
    path = np.array([[p[1], p[0]] for p in path])
    ##########################
    # Start the robot movement loop
    while True:
        acc = thytanic.read_accelerometer()
        if(acc[2] > 28):
            print("kidnapping initiated:", acc[2])
            thytanic.set_wheel_speed(0, 0)
            time.sleep(10)
            #Recomputing new position and path
            img, thymiopos, initital_angle = get_current_state(cam=cam,
                obstacle_range=obstacle_range,
                target_range=target_range,
                th_front_range=th_front_range,
                th_back_range=th_back_range,
            )
            endsize = [120, 160]

            #this line generates the original enposition, thymio position, and the obstacle channel
            thymiopos, endpos, obstacleimage = downsamplingprep(img, endsize, 1, 1)

            #this line turns obstacleimage into its downsampled version with enlarged obstacles
            preppedimage = downsampling(obstacleimage, endsize, 25, 0.2)
            path, exploredpoints, unexploredpoints = pathfinder(thymiopos, endpos, preppedimage)  

            # Set the robot to GLOBAL_MOVEMENT state
            print("initital_angle:", initital_angle)
            thytanic.x_est = np.array([thymiopos[1], 0, thymiopos[0], 0,initital_angle,0]) #position, velocity (x,y,angle)
            thytanic.P_est = np.eye(6) # TO TUNE
            thytanic.goal_idx = 0
            thytanic.robot_state = ThytanicState.GLOBAL_MOVEMENT
            #thytanic.set_wheel_speed(100, 100)
            print("Thytanic state set to GLOBAL_MOVEMENT.")
            x_estimated = []
            ########################
            #new path
            path = path[::5]
            path = path[2:]
            path = np.array([[p[1], p[0]] for p in path])  
        _, thymiopos, orientation = get_current_state(    cam=cam,
            obstacle_range=obstacle_range,
            target_range=target_range,
            th_front_range=th_front_range,
            th_back_range=th_back_range,
        )
        # apply kalman filter and get the estimated positions thytanic.x_est
        kalman_filter(thytanic, [thymiopos[1], thymiopos[0], orientation])
        x_estimated.append(thytanic.x_est)
        #print("x_est", thytanic.x_est)
        thytanic.goal = path[thytanic.goal_idx] # get the (x,y) coordinates of the goal point
        print("goal", thytanic.goal)
        #print("goal_idx", thytanic.goal_idx)
        thytanic.update_robot_state()

        # Print the proximity sensor values
        sensor_values = thytanic.read_proximity_sensors()
        #print("Proximity sensor readings -- threshold of 2000 for detection:", sensor_values)
        if(not fig == 0):
            fig.clear()
        fig = plt.figure()
        plt.plot(np.array(x_estimated)[:,0], np.array(x_estimated)[:,2], label='Estimated')
        plt.plot(path[:,0], path[:,1], label='path')
        # Plot the path
        plt.legend()
        plt.show()
        if thytanic.goal_idx >= len(path):
            thytanic.robot_state = ThytanicState.STOP
            thytanic.update_robot_state()
            print("Goal reached!")
            break 

except KeyboardInterrupt:
    # Handle interruption and stop the robot safely
    print("\nStopping the Thytanic...")
    thytanic.disconnect()
    print("Thytanic stopped and disconnected.")

finally:
    print("Session ended.")
    plt.figure()
    plt.plot(np.array(x_estimated)[:,0], np.array(x_estimated)[:,2], label='Estimated')
    plt.plot(path[:,0], path[:,1], label='path')
    # Plot the path
    plt.legend()
    thytanic.disconnect()

Connecting to Thytanic...
Connection established!
initital_angle: 1.0104855546316993
Thytanic state set to GLOBAL_MOVEMENT.
accelerometer: 0
accelerometer: 0
accelerometer: 20
goal [117  71]
x 116.4025848008295 y 80.09922871727893 theta 1.01406473925073
goal index 0
delta_x 0.5974151991704986 delta_y -9.099228717278933
alpha 0.49117009723007365
v 70.73089118344487 omega 17.68212350028265
left speed 28.420017185728472
right speed 108.41057587748331


<IPython.core.display.Javascript object>

accelerometer: 0
accelerometer: 0
accelerometer: 20
goal [117  71]
x 116.38918372491824 y 80.06568748642923 theta 0.9972908090908386
goal index 0
delta_x 0.6108162750817598 delta_y -9.065687486429226
alpha 0.5062304772054118
v 70.61636337874506 omega 18.224297179394824
left speed 27.082893576643094
right speed 109.52614272152444


<IPython.core.display.Javascript object>

accelerometer: 0
accelerometer: 0
accelerometer: 20
goal [117  71]
x 116.59020682026198 y 79.21232164824288 theta 1.1109472715445061
goal index 0
delta_x 0.409793179738017 delta_y -8.212321648242877
alpha 0.40999061355881206
v 67.42013164389203 omega 14.759662088117235
left speed 31.827974991951823
right speed 98.59787491438695


...

accelerometer: 0
accelerometer: 0
accelerometer: 19
goal [144  43]
x 140.66450481567784 y 48.34101011115368 theta 0.8777081194128095
goal index 11
delta_x 3.3354951843221556 delta_y -5.341010111153679
alpha 0.13484389385043227
v 58.88222877261704 omega 4.854380178615562


<IPython.core.display.Javascript object>

accelerometer: 0
accelerometer: 0
accelerometer: 20
goal [147  41]
x 141.65039330756906 y 47.44844995684178 theta 0.8798177205246757
goal index 12
delta_x 5.349606692430939 delta_y -6.4484499568417775
alpha -0.001549825891481449
v 50 omega -0.05579373209333216
left speed 48.48929534640158
right speed 48.23689512978889

Stopping the Thytanic...
Thytanic stopped and disconnected.
Session ended.


<IPython.core.display.Javascript object>

AttributeError: 'NoneType' object has no attribute 'set_variables'

## 9. Conclusion